In [1]:
import jax 
import jax.numpy as jnp
import flax.linen as nn

In [35]:
shared_params = False
if shared_params:
    BatchDense = nn.vmap(nn.Dense, in_axes=0, out_axes=0, variable_axes={'params': None}, split_rngs={'params': False})
else:
    BatchDense = nn.vmap(nn.Dense, in_axes=0, out_axes=0, variable_axes={'params': 0}, split_rngs={'params': True})

In [36]:
batch_size = 10
embedding_dim = 5
x = jnp.ones((batch_size, embedding_dim))
dense = BatchDense(features=3)
params = dense.init(jax.random.PRNGKey(0), x)

In [37]:
# write a function that prints recursively the keys and shapes of a nested dictionary
def map_nested_fn(fn):
    """Recursively apply `fn to the key-value pairs of a nested dict / pytree."""

    def map_fn(k_in, nested_dict):
        if k_in != None: print(f'{k_in}', end=' ')
        return {
            k: (map_fn(k, v) if hasattr(v, "keys") else fn(k, v))
            for k, v in nested_dict.items()
        }
    
    return map_fn

map_nested_fn(lambda k, v: print(f'\n {k, v.shape}'))(None, params)

params 
 ('bias', (10, 3))

 ('kernel', (10, 5, 3))


{'params': {'bias': None, 'kernel': None}}

In [38]:
y = dense.apply(params, x)

In [39]:
y

Array([[ 1.9910165 ,  0.28383535, -1.047992  ],
       [ 1.3863918 ,  0.5475614 ,  1.2088555 ],
       [-1.3427639 , -0.58169377, -0.02100402],
       [-1.6752058 , -0.68735284, -1.4142416 ],
       [ 0.7120553 ,  0.13763508, -0.47188094],
       [ 0.59918714, -0.13747637,  0.40842372],
       [-0.18032235, -0.48730588, -1.0494046 ],
       [-0.04323477, -0.4011463 ,  0.63187987],
       [ 0.50989723,  0.20760426,  0.13128324],
       [ 0.53805596, -1.6646125 ,  1.1062521 ]], dtype=float32)

In [44]:
def step(x_k_1, u_k):
    x_k = x_k_1 + u_k
    y_k = jnp.tanh(x_k)
    return x_k, y_k
x_last, y_entire = jax.lax.scan(step, 0, jnp.ones((5,)))

In [46]:
x_last

Array(5., dtype=float32)

In [47]:
y_entire

Array([0.7615942 , 0.9640276 , 0.9950547 , 0.9993292 , 0.99990916],      dtype=float32)

In [53]:
print([float(jnp.tanh(i)) for i in range(5)])

[0.0, 0.7615941762924194, 0.9640275835990906, 0.9950547218322754, 0.9993292093276978]


In [ ]:
# q: what does u[:, jnp.newaxis] do?
# a: it adds a new axis at the end of the array

In [2]:
!pip install einops


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [19]:
import torch
from einops import repeat
import math
H=3
N=4
log_A_real = torch.log(0.5 * torch.ones(H, N))
A_imag = math.pi * repeat(torch.arange(N), 'n -> h n', h=H)


In [20]:
A_imag

tensor([[0.0000, 3.1416, 6.2832, 9.4248],
        [0.0000, 3.1416, 6.2832, 9.4248],
        [0.0000, 3.1416, 6.2832, 9.4248]])

In [21]:
log_A_real

tensor([[-0.6931, -0.6931, -0.6931, -0.6931],
        [-0.6931, -0.6931, -0.6931, -0.6931],
        [-0.6931, -0.6931, -0.6931, -0.6931]])

In [22]:
A = log_A_real + 1j * A_imag

In [23]:
A

tensor([[-0.6931+0.0000j, -0.6931+3.1416j, -0.6931+6.2832j, -0.6931+9.4248j],
        [-0.6931+0.0000j, -0.6931+3.1416j, -0.6931+6.2832j, -0.6931+9.4248j],
        [-0.6931+0.0000j, -0.6931+3.1416j, -0.6931+6.2832j, -0.6931+9.4248j]])

In [24]:
C = torch.randn(H, N // 2, dtype=torch.cfloat)
Cb = torch.view_as_real(C)
print(Cb.shape)

torch.Size([3, 4, 2])


In [25]:
Cbb = torch.view_as_complex(Cb)
print(Cbb.shape)

torch.Size([3, 4])


In [26]:
dt_min = 1e-3
dt_max = 1e-1
log_dt = torch.rand(H) * (
            math.log(dt_max) - math.log(dt_min)
        ) + math.log(dt_min)


In [27]:
dt = torch.exp(log_dt)
dt

tensor([0.0347, 0.0453, 0.0606])

In [30]:
dt.unsqueeze(-1)

tensor([[0.0347],
        [0.0453],
        [0.0606]])

In [28]:
dtA = A * dt.unsqueeze(-1)
dtA

tensor([[-0.0241+0.0000j, -0.0241+0.1091j, -0.0241+0.2183j, -0.0241+0.3274j],
        [-0.0314+0.0000j, -0.0314+0.1424j, -0.0314+0.2848j, -0.0314+0.4272j],
        [-0.0420+0.0000j, -0.0420+0.1905j, -0.0420+0.3809j, -0.0420+0.5714j]])

In [29]:
3.1416 * 0.0347

0.10901352

In [40]:
L = 4
dtA.unsqueeze(-1) * torch.arange(L, device=A.device)

tensor([[[-0.0000+0.0000j, -0.0241+0.0000j, -0.0482+0.0000j, -0.0722+0.0000j],
         [-0.0000+0.0000j, -0.0241+0.1091j, -0.0482+0.2183j, -0.0722+0.3274j],
         [-0.0000+0.0000j, -0.0241+0.2183j, -0.0482+0.4366j, -0.0722+0.6549j],
         [-0.0000+0.0000j, -0.0241+0.3274j, -0.0482+0.6549j, -0.0722+0.9823j]],

        [[-0.0000+0.0000j, -0.0314+0.0000j, -0.0628+0.0000j, -0.0942+0.0000j],
         [-0.0000+0.0000j, -0.0314+0.1424j, -0.0628+0.2848j, -0.0942+0.4272j],
         [-0.0000+0.0000j, -0.0314+0.2848j, -0.0628+0.5696j, -0.0942+0.8543j],
         [-0.0000+0.0000j, -0.0314+0.4272j, -0.0628+0.8543j, -0.0942+1.2815j]],

        [[-0.0000+0.0000j, -0.0420+0.0000j, -0.0840+0.0000j, -0.1261+0.0000j],
         [-0.0000+0.0000j, -0.0420+0.1905j, -0.0840+0.3809j, -0.1261+0.5714j],
         [-0.0000+0.0000j, -0.0420+0.3809j, -0.0840+0.7618j, -0.1261+1.1428j],
         [-0.0000+0.0000j, -0.0420+0.5714j, -0.0840+1.1428j, -0.1261+1.7141j]]])

In [32]:
a = torch.ones(2,2)
a.unsqueeze(-1) * torch.arange(3)

tensor([[[0., 1., 2.],
         [0., 1., 2.]],

        [[0., 1., 2.],
         [0., 1., 2.]]])

In [38]:
b = torch.tensor([[1,2],[3,4]])
b.unsqueeze(-1)

tensor([[[1],
         [2]],

        [[3],
         [4]]])

In [39]:
b.unsqueeze(-1) * torch.arange(3)

tensor([[[0, 1, 2],
         [0, 2, 4]],

        [[0, 3, 6],
         [0, 4, 8]]])

In [58]:
import torch
from einops import repeat
import math
H=5
N=4
L=3
dt=0.1
log_A_real = torch.log(0.5 * torch.ones(N,))
A_imag = math.pi * torch.arange(N)
print(f'{log_A_real=}')
print(f'{A_imag=}')


log_A_real=tensor([-0.6931, -0.6931, -0.6931, -0.6931])
A_imag=tensor([0.0000, 3.1416, 6.2832, 9.4248])


In [59]:
A = -torch.exp(log_A_real) + 1j * A_imag
A

tensor([-0.5000+0.0000j, -0.5000+3.1416j, -0.5000+6.2832j, -0.5000+9.4248j])

In [60]:
dtA = A * dt
dtA

tensor([-0.0500+0.0000j, -0.0500+0.3142j, -0.0500+0.6283j, -0.0500+0.9425j])

In [61]:
dtA_unsqueeze = dtA.unsqueeze(-1)
dtA_unsqueeze

tensor([[-0.0500+0.0000j],
        [-0.0500+0.3142j],
        [-0.0500+0.6283j],
        [-0.0500+0.9425j]])

In [63]:
K = dtA_unsqueeze * torch.arange(L)
K

tensor([[-0.0000+0.0000j, -0.0500+0.0000j, -0.1000+0.0000j],
        [-0.0000+0.0000j, -0.0500+0.3142j, -0.1000+0.6283j],
        [-0.0000+0.0000j, -0.0500+0.6283j, -0.1000+1.2566j],
        [-0.0000+0.0000j, -0.0500+0.9425j, -0.1000+1.8850j]])

In [65]:
exp_K = torch.exp(K)
exp_K

tensor([[ 1.0000+0.0000j,  0.9512+0.0000j,  0.9048+0.0000j],
        [ 1.0000+0.0000j,  0.9047+0.2939j,  0.7320+0.5319j],
        [ 1.0000+0.0000j,  0.7696+0.5591j,  0.2796+0.8606j],
        [ 1.0000+0.0000j,  0.5591+0.7696j, -0.2796+0.8606j]])

In [70]:
C = torch.randn(N, dtype=torch.cfloat)
C

tensor([-0.5039+0.6777j, -1.7526-0.1343j, -0.7315+0.6349j,  1.1549-0.3224j])

In [74]:
K_end = C @ exp_K
K_end

tensor([-1.8330+0.8559j, -2.0494+0.7961j, -2.4639+0.2148j])

In [84]:
import jax
import jax.numpy as jnp
# set display to .4f in jax 
def initialization(N: int) -> torch.Tensor:
    log_A_real = torch.log(0.5 * torch.ones(N,))
    A_imag = math.pi * torch.arange(N)
    C = torch.randn(N, dtype=torch.cfloat)
    return log_A_real, A_imag, C

def init_jax(key, N: int) -> jnp.ndarray:
    log_A_real = jnp.log(0.5 * jnp.ones(N,))
    A_imag = math.pi * jnp.arange(N)
    C = jax.random.normal(key, (N,), dtype=jnp.complex64)
    return log_A_real, A_imag, C

# fix the seed
torch.manual_seed(0)
log_A_real, A_imag, C = initialization(N=4)
log_A_real, A_imag, C

(tensor([-0.6931, -0.6931, -0.6931, -0.6931]),
 tensor([0.0000, 3.1416, 6.2832, 9.4248]),
 tensor([ 1.0896-0.2075j, -1.5406+0.4019j, -0.7669-0.9890j,  0.2852+0.5926j]))

In [85]:
log_A_real_jax, A_imag_jax, C_jax = init_jax(jax.random.PRNGKey(0), N=4)
log_A_real_jax, A_imag_jax, C_jax

(Array([-0.6931472, -0.6931472, -0.6931472, -0.6931472], dtype=float32),
 Array([0.       , 3.1415927, 6.2831855, 9.424778 ],      dtype=float32, weak_type=True),
 Array([ 0.09823954+0.8046016j ,  0.9692087 -0.10133854j,
        -0.3755876 -0.41827938j,  0.01438101+0.5619111j ], dtype=complex64))

In [86]:
def kernel(log_A_real, A_imag, C, dt, L):
    A = -torch.exp(log_A_real) + 1j * A_imag
    dtA = A * dt
    K = dtA.unsqueeze(-1) * torch.arange(L)
    exp_K = torch.exp(K)
    K_end = C @ exp_K
    return K_end

def kernel_jax(log_A_real, A_imag, C, dt, L):
    A = -jnp.exp(log_A_real) + 1j * A_imag
    dtA = A * dt
    K = dtA[:, jnp.newaxis] * jnp.arange(L)
    exp_K = jnp.exp(K)
    K_end = C @ exp_K
    return K_end

In [87]:
import time

In [95]:
%timeit kernel(log_A_real, A_imag, C, 0.1, L=3)

18.9 µs ± 144 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [89]:
%timeit kernel_jax(log_A_real_jax, A_imag_jax, C_jax, 0.1, 3)

160 µs ± 19.9 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [92]:
jit_kernel_jax = jax.jit(kernel_jax, static_argnums=(3, 4))
# warmup 
jit_kernel_jax(log_A_real_jax, A_imag_jax, C_jax, 0.1, 3)

Array([0.70624167+0.84689474j, 0.5204996 +0.75193024j,
       0.6196348 +0.5844165j ], dtype=complex64)

In [94]:
%timeit jit_kernel_jax(log_A_real_jax, A_imag_jax, C_jax, 0.1, L=3)

4.11 µs ± 935 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [132]:
log_A_real_list = []
A_imag_list = []
C_list = []
H = 10
N = 64
key = jax.random.PRNGKey(0)
for i in range(H):
    key, subkey = jax.random.split(key)
    log_A_real, A_imag, C = init_jax(subkey, N=N)
    log_A_real_list.append(log_A_real)
    A_imag_list.append(A_imag)
    C_list.append(C)
log_A_real_batch = jnp.stack(log_A_real_list)
A_imag_batch = jnp.stack(A_imag_list)
C_batch = jnp.stack(C_list)
print(f'{log_A_real_batch.shape=}')
print(f'{A_imag_batch.shape=}')
print(f'{C_batch.shape=}')
print(log_A_real_batch + 1j * A_imag_batch)

log_A_real_batch.shape=(10, 64)
A_imag_batch.shape=(10, 64)
C_batch.shape=(10, 64)
[[-0.6931472  +0.j        -0.6931472  +3.1415927j -0.6931472  +6.2831855j
  -0.6931472  +9.424778j  -0.6931472 +12.566371j  -0.6931472 +15.707964j
  -0.6931472 +18.849556j  -0.6931472 +21.99115j   -0.6931472 +25.132742j
  -0.6931472 +28.274334j  -0.6931472 +31.415928j  -0.6931472 +34.55752j
  -0.6931472 +37.699112j  -0.6931472 +40.840706j  -0.6931472 +43.9823j
  -0.6931472 +47.12389j   -0.6931472 +50.265484j  -0.6931472 +53.407078j
  -0.6931472 +56.548668j  -0.6931472 +59.69026j   -0.6931472 +62.831856j
  -0.6931472 +65.97345j   -0.6931472 +69.11504j   -0.6931472 +72.25663j
  -0.6931472 +75.398224j  -0.6931472 +78.53982j   -0.6931472 +81.68141j
  -0.6931472 +84.823006j  -0.6931472 +87.9646j    -0.6931472 +91.106186j
  -0.6931472 +94.24778j   -0.6931472 +97.38937j   -0.6931472+100.53097j
  -0.6931472+103.67256j   -0.6931472+106.814156j  -0.6931472+109.95575j
  -0.6931472+113.097336j  -0.6931472+116.23893j

In [136]:
vmap_kernel_jax = jax.vmap(jit_kernel_jax, in_axes=(0, 0, 0, None, None))
# warmup
vmap_kernel_jax(log_A_real_batch, A_imag_batch, C_batch, 0.1, 128)

Array([[ 4.93954849e+00-7.8557873e-01j,  1.10475659e+00-1.1616502e+01j,
        -1.03798866e-01+4.1941798e-01j, ...,
        -7.51979509e-03-1.0882646e-02j,  6.20917045e-03-1.8583246e-02j,
         1.21726161e-02-1.3888414e-02j],
       [-4.04846907e-01+8.7479954e+00j,  5.16121721e+00-2.3994251e-01j,
        -2.49888492e+00-2.6765285e+00j, ...,
         1.91112496e-02-1.3435737e-03j,  1.23324292e-03+3.1608322e-03j,
         4.53132531e-03-9.6221790e-03j],
       [-6.86919308e+00+1.1445875e+01j, -3.01707697e+00-1.9842070e+01j,
         3.69161034e+00+4.1499758e+00j, ...,
         9.38161649e-03-8.1465840e-03j, -1.20261209e-02+9.6609816e-03j,
        -1.32796876e-02-4.7527147e-03j],
       ...,
       [-2.59423447e+00+6.4350688e-01j, -2.92640305e+00-9.5094881e+00j,
        -4.76090574e+00-1.9538455e+00j, ...,
         1.01800896e-02-1.5535339e-02j, -2.54037790e-04-2.9856451e-03j,
        -1.15122208e-02-4.4159214e-03j],
       [-4.17223358e+00-4.4832230e-01j, -7.96455956e+00+7.3788130e-0

In [137]:
%timeit vmap_kernel_jax(log_A_real_batch, A_imag_batch, C_batch, 0.1, 3)

443 µs ± 2.32 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [138]:
def batch_init_torch(H, N):
    log_A_real = torch.log(0.5 * torch.ones(H, N))
    A_imag = math.pi * repeat(torch.arange(N), 'n -> h n', h=H)
    C = torch.randn(H, N, dtype=torch.cfloat)
    return log_A_real, A_imag, C

batch_log_A_real_torch, batch_A_imag_torch, batch_C_torch = batch_init_torch(H=10, N=4)
print(f'{batch_log_A_real_torch.shape=}')
print(f'{batch_A_imag_torch.shape=}')
print(f'{batch_C_torch.shape=}')
print(log_A_real_batch + 1j * A_imag_batch)

batch_log_A_real_torch.shape=torch.Size([10, 4])
batch_A_imag_torch.shape=torch.Size([10, 4])
batch_C_torch.shape=torch.Size([10, 4])
[[-0.6931472  +0.j        -0.6931472  +3.1415927j -0.6931472  +6.2831855j
  -0.6931472  +9.424778j  -0.6931472 +12.566371j  -0.6931472 +15.707964j
  -0.6931472 +18.849556j  -0.6931472 +21.99115j   -0.6931472 +25.132742j
  -0.6931472 +28.274334j  -0.6931472 +31.415928j  -0.6931472 +34.55752j
  -0.6931472 +37.699112j  -0.6931472 +40.840706j  -0.6931472 +43.9823j
  -0.6931472 +47.12389j   -0.6931472 +50.265484j  -0.6931472 +53.407078j
  -0.6931472 +56.548668j  -0.6931472 +59.69026j   -0.6931472 +62.831856j
  -0.6931472 +65.97345j   -0.6931472 +69.11504j   -0.6931472 +72.25663j
  -0.6931472 +75.398224j  -0.6931472 +78.53982j   -0.6931472 +81.68141j
  -0.6931472 +84.823006j  -0.6931472 +87.9646j    -0.6931472 +91.106186j
  -0.6931472 +94.24778j   -0.6931472 +97.38937j   -0.6931472+100.53097j
  -0.6931472+103.67256j   -0.6931472+106.814156j  -0.6931472+109.955

In [139]:
def batch_kernel(log_A_real, A_imag, C, dt, L):
    A = -torch.exp(log_A_real) + 1j * A_imag
    dtA = A * dt
    K = dtA.unsqueeze(-1) * torch.arange(L)
    #print(f'{K.shape=}')
    exp_K = torch.exp(K)
    K_end = torch.einsum('hn, hnl -> hl', C, exp_K)
    return K_end

In [144]:
%timeit batch_kernel(batch_log_A_real_torch, batch_A_imag_torch, batch_C_torch, 0.1, 128)

115 µs ± 18.3 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [145]:
batch_kernel(batch_log_A_real_torch, batch_A_imag_torch, batch_C_torch, 0.1, 128)

tensor([[-4.3008e-01+1.2949e+00j, -4.5447e-01+1.4629e+00j,
         -8.6903e-01+1.5014e+00j,  ...,
         -2.9321e-03-7.6225e-04j, -1.3982e-03-1.4989e-03j,
         -1.0464e-04-1.1779e-03j],
        [ 1.3262e+00+1.5765e-02j,  1.4889e+00+1.5837e+00j,
          3.6505e-01+2.8774e+00j,  ...,
         -7.5105e-03+7.7568e-04j, -5.6297e-03-2.1715e-03j,
         -2.9498e-03-2.9153e-03j],
        [ 1.8445e+00+1.3649e+00j,  1.5746e+00+1.4333e+00j,
          1.2392e+00+1.4710e+00j,  ...,
          3.6131e-04+1.1230e-03j,  7.9403e-04-4.7692e-04j,
          2.1474e-03-1.4642e-03j],
        ...,
        [-5.7233e-01-2.4335e+00j,  1.1044e+00-6.3042e-01j,
          6.8477e-01+1.4851e+00j,  ...,
         -6.5966e-03-1.6509e-03j, -3.1269e-03-4.3812e-03j,
          9.2000e-04-3.4647e-03j],
        [ 4.8408e-01-1.6851e+00j,  7.5634e-01-4.6490e-01j,
         -2.0635e-02+3.2449e-01j,  ...,
         -8.6199e-04-4.8899e-03j,  2.6942e-03-4.5556e-03j,
          5.0336e-03-1.5003e-03j],
        [-1.2433e+00+1

In [146]:
def batch_kernel_jax(log_A_real, A_imag, C, dt, L):
    A = -jnp.exp(log_A_real) + 1j * A_imag
    dtA = A * dt
    K = dtA[..., jnp.newaxis] * jnp.arange(L)
    exp_K = jnp.exp(K)
    K_end = jnp.einsum('hn, hnl -> hl', C, exp_K)
    return K_end

In [150]:
jit_batch_kernel_jax = jax.jit(batch_kernel_jax, static_argnums=(3, 4))
# warmup
jit_batch_kernel_jax(log_A_real_batch, A_imag_batch, C_batch, 0.1, 128)

Array([[ 4.93954849e+00-7.8557873e-01j,  1.10475659e+00-1.1616502e+01j,
        -1.03798866e-01+4.1941798e-01j, ...,
        -7.51979509e-03-1.0882646e-02j,  6.20917045e-03-1.8583246e-02j,
         1.21726161e-02-1.3888414e-02j],
       [-4.04846907e-01+8.7479954e+00j,  5.16121721e+00-2.3994251e-01j,
        -2.49888492e+00-2.6765285e+00j, ...,
         1.91112496e-02-1.3435737e-03j,  1.23324292e-03+3.1608322e-03j,
         4.53132531e-03-9.6221790e-03j],
       [-6.86919308e+00+1.1445875e+01j, -3.01707697e+00-1.9842070e+01j,
         3.69161034e+00+4.1499758e+00j, ...,
         9.38161649e-03-8.1465840e-03j, -1.20261209e-02+9.6609816e-03j,
        -1.32796876e-02-4.7527147e-03j],
       ...,
       [-2.59423447e+00+6.4350688e-01j, -2.92640305e+00-9.5094881e+00j,
        -4.76090574e+00-1.9538455e+00j, ...,
         1.01800896e-02-1.5535339e-02j, -2.54037790e-04-2.9856451e-03j,
        -1.15122208e-02-4.4159214e-03j],
       [-4.17223358e+00-4.4832230e-01j, -7.96455956e+00+7.3788130e-0

In [151]:
%timeit jit_batch_kernel_jax(log_A_real_batch, A_imag_batch, C_batch, 0.1, 128)

425 µs ± 211 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [152]:
!pip freeze | grep torch

torch==2.1.2


In [156]:
torch.mps.

TypeError: 'module' object is not iterable

In [158]:
# Check that MPS is available
if not torch.backends.mps.is_available():
    if not torch.backends.mps.is_built():
        print("MPS not available because the current PyTorch install was not "
              "built with MPS enabled.")
    else:
        print("MPS not available because the current MacOS version is not 12.3+ "
              "and/or you do not have an MPS-enabled device on this machine.")

else:
    print("GOOD NEWS! MPS is available on this machine.")
    mps_device = torch.device("mps")

    # Create a Tensor directly on the mps device
    x = torch.ones(5, device=mps_device)
    # Or
    x = torch.ones(5, device="mps")

    # Any operation happens on the GPU
    y = x * 2
    print(y.device)

GOOD NEWS! MPS is available on this machine.
mps:0
